# Imports

In [1]:
import pandas as pd
import boto3
import json
import configparser

In [2]:
from IPython.display import JSON

In [3]:
import getpass

# Configuration variables

In [4]:
# AWS credentials
AWS_KEY                = None
AWS_SECRET             = None
AWS_REGION             = None

# Cluster configuration
DWH_CLUSTER_TYPE       = None
DWH_NUM_NODES          = None
DWH_NODE_TYPE          = None

# Redshift details
DWH_CLUSTER_IDENTIFIER = None
DWH_DB                 = None
DWH_DB_USER            = None
DWH_DB_PASSWORD        = None
DWH_PORT               = None

# IAM role
DWH_IAM_ROLE_NAME      = None

In [5]:
AWS_SECRET = getpass.getpass("Please give your secret key")

# Parse config file

In [7]:
config = configparser.ConfigParser()

with open('/home/workspace/dev/dwh.cfg') as fp:
    config.read_file(fp)
    AWS_KEY                = config.get('AWS','KEY')
#     AWS_SECRET             = config.get('AWS','SECRET')
    AWS_REGION             = config.get('AWS','REGION')
    DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
    DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
    DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
    DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
    DWH_DB                 = config.get("DWH","DWH_DB")
    DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
    DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
    DWH_PORT               = config.get("DWH","DWH_PORT")
    DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

# Remove IAM Role 

In [8]:
from botocore.exceptions import ClientError

In [9]:
aws_iam = boto3.client(
    'iam',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET,
)

In [10]:
aws_iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")

{'ResponseMetadata': {'RequestId': '92d20a4d-9995-4b69-93cb-f6f20fc5f7fc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '92d20a4d-9995-4b69-93cb-f6f20fc5f7fc',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 03 Sep 2020 13:40:12 GMT'},
  'RetryAttempts': 0}}

In [11]:
aws_iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)

{'ResponseMetadata': {'RequestId': 'd43370ac-0132-4302-bd9f-01589ef02bc1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd43370ac-0132-4302-bd9f-01589ef02bc1',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Thu, 03 Sep 2020 13:40:12 GMT'},
  'RetryAttempts': 0}}

# Remove Redshift cluster

In [12]:
aws_redshift = boto3.client(
    'redshift',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)

In [13]:
clusterProps = aws_redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [14]:
JSON(clusterProps, expanded=True)

<IPython.core.display.JSON object>

In [15]:
aws_redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'dendcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'de_nd_user',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dendcluster.cwlrei9kynlz.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2020, 9, 3, 11, 37, 26, 682000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-741d325c',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-9dbefae5',
  'AvailabilityZone': 'us-west-2d',
  'PreferredMaintenanceWindow': 'fri:08:30-fri:09:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
 

# Remove EC2 VPC

In [16]:
DWH_ENDPOINT = clusterProps['Endpoint']['Address']
DWH_ROLE_ARN = clusterProps['IamRoles'][0]['IamRoleArn']

In [17]:
print(f"Endpoint: {DWH_ENDPOINT}")

Endpoint: dendcluster.cwlrei9kynlz.us-west-2.redshift.amazonaws.com


In [18]:
print(f"Role ARN: {DWH_ROLE_ARN}")

Role ARN: arn:aws:iam::498285492063:role/aws_iam_redshift


In [19]:
aws_ec2 = boto3.resource(
    'ec2',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)

In [20]:
try:
    vpc = aws_ec2.Vpc(id=clusterProps['VpcId'])
    
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.revoke_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
    
except Exception as err:
    print(err)

ec2.SecurityGroup(id='sg-741d325c')
